In [44]:
# # ChromaDB Admin Interface
# Админские функции для управления векторной базой

In [45]:
import os
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd

from dotenv import load_dotenv

from chromadb import Client, Settings
from chromadb.utils import embedding_functions

from sentence_transformers import SentenceTransformer

from IPython.display import display
import ipywidgets as widgets

In [46]:
# Настройки
DB_PATH = "./db"
COLLECTION_NAME = "docs"
EMBEDDING_MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
EMBEDDING_DIM = 768

In [47]:
# Загрузка конфигурации
load_dotenv()

# Модель для эмбеддингов
embedder = SentenceTransformer(EMBEDDING_MODEL)
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=EMBEDDING_MODEL)

# Функция для нормализации эмбеддингов
def normalize_embedding(embedding):
    """Normalize embedding to unit length (L2 norm = 1)."""
    embedding = np.array(embedding)
    norm = np.linalg.norm(embedding)
    if norm > 0:
        return (embedding / norm).tolist()
    return embedding.tolist()

# Инициализация ChromaDB
db = Client(settings=Settings(
    persist_directory="./db",
    allow_reset=True,
    is_persistent=True  # Автосохранение включено
))
collection = db.get_or_create_collection(
    name="docs",
    embedding_function=embedding_func
)

In [48]:
# Проверка подключения
print(f"Доступные коллекции: {db.list_collections()}")
print(f"Используемая коллекция: {collection.name}")

Доступные коллекции: [Collection(name=docs)]
Используемая коллекция: docs


In [49]:
# Основные функции администрирования
def show_collection_stats():
    """Показать статистику коллекции"""
    count = collection.count()
    stats = {
        "Количество документов": count,
        "Размерность эмбеддингов": len(collection.peek()["embeddings"][0]) if count > 0 else 0,
        "Уникальных пользователей": len(set(m.get("user", "unknown") for m in collection.get()["metadatas"])) if count > 0 else 0,
        "Средняя длина документа": np.mean([len(doc) for doc in collection.get()["documents"]]) if count > 0 else 0
    }
    print("\n".join(f"{k}: {v:.2f}" if isinstance(v, float) else f"{k}: {v}" for k, v in stats.items()))

def search_documents(query: str, n_results: int = 5, similarity_threshold: float = 0.5):
    """Семантический поиск с фильтрацией"""
    results = collection.query(
        query_texts=[query],
        n_results=n_results,
        include=["documents", "metadatas", "distances"]
    )
    
    filtered = [
        (doc, meta, 1-dist, id)
        for doc, meta, dist, id in zip(
            results["documents"][0],
            results["metadatas"][0],
            results["distances"][0],
            results["ids"][0]
        )
        if (1 - dist) >= similarity_threshold
    ]
    
    return pd.DataFrame(filtered, columns=["Текст", "Метаданные", "Схожесть", "ID"])

def export_to_dataframe():
    """Экспорт всей коллекции в DataFrame с эмбеддингами"""
    data = collection.get(include=["documents", "metadatas", "embeddings"])
    
    # Преобразуем эмбеддинги в список строк для Pandas
    embeddings = [emb for emb in data["embeddings"]] if (data["embeddings"] is not None and len(data["embeddings"]) > 0) else [None]*len(data["ids"])
    
    return pd.DataFrame({
        "ids": data["ids"],
        "documents": data["documents"],
        "metadatas": data["metadatas"],
        "embeddings": embeddings
    })

def add_document(text: str, metadata: dict = {}):
    """Добавить документ в коллекцию"""
    embedding = normalize_embedding(embedder.encode(text).tolist())
    
    if 'source' not in metadata:
        metadata['source'] = 'Admin_Notebook'
    if 'timestamp' not in metadata:
        metadata['timestamp'] = datetime.now().isoformat()
    if 'user' not in metadata:
        metadata['user'] = 'Admin'

    user = metadata['user']

    doc_id = f"doc_{user}_{datetime.now().timestamp()}"

    collection.add(
        documents=[text],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[doc_id]
    )
    print(f"Документ {doc_id} добавлен")

def delete_document(doc_id: str):
    """Удалить документ по ID"""
    if not collection.get(ids=[doc_id])["ids"]:
        print(f"Документ {doc_id} не найден")
        return
    collection.delete(ids=[doc_id])
    print(f"Документ {doc_id} удален")

def update_document(doc_id: str, new_text: str, new_metadata: dict = {}):
    """Обновить документ"""
    if not collection.get(ids=[doc_id])["ids"]:
        print(f"Документ {doc_id} не найден")
        return
    new_embedding = normalize_embedding(embedder.encode(new_text).tolist())
    
    if 'source' not in new_metadata:
        new_metadata['source'] = 'Admin_Notebook'
    if 'timestamp' not in new_metadata:
        new_metadata['timestamp'] = datetime.now().isoformat()
    if 'user' not in new_metadata:
        new_metadata['user'] = 'Admin'

    collection.update(
        ids=[doc_id],
        documents=[new_text],
        embeddings=[new_embedding],
        metadatas=[new_metadata]
    )
    print(f"Документ {doc_id} обновлен")

def clean_collection():
    """Очистка коллекции с подтверждением"""
    if collection.count() == 0:
        print("Коллекция уже пуста!")
        return
    print(f"Будет удалено {collection.count()} документов")
    while True:
        response = input("Подтвердите очистку (y/n): ").lower().strip()
        if response in ['y', 'n']:
            break
        print("Пожалуйста, введите 'y' или 'n'")
    if response == 'y':
        collection.delete(ids=collection.get()["ids"])
        print("Коллекция очищена")
    else:
        print("Отменено")

def backup_collection():
    """Создание резервной копии"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    backup_dir = "./reserve_copies"
    pathlib.Path(backup_dir).mkdir(exist_ok=True)
    backup_file = f"{backup_dir}/backup_{timestamp}.parquet"
    df = export_to_dataframe()
    df.to_parquet(backup_file)
    print(f"Резервная копия сохранена в {backup_file}")

def restore_from_backup(backup_file: str):
    """Восстановление коллекции из резервной копии"""
    if not os.path.exists(backup_file):
        print(f"Файл {backup_file} не найден")
        return
    df = pd.read_parquet(backup_file)
    if df.empty:
        print("Резервная копия пуста")
        return
    collection.delete(ids=collection.get()["ids"])  # Очистка текущей коллекции
    collection.add(
        ids=df["ids"].tolist(),
        documents=df["documents"].tolist(),
        embeddings=df["embeddings"].tolist(),
        metadatas=df["metadatas"].tolist()
    )
    print(f"Коллекция восстановлена из {backup_file}")

In [50]:
# можно сделать класс
class ChromaAdmin:
    def __init__(self, db_path, collection_name):
        self.client = Client(settings=Settings(
            persist_directory=db_path,
            is_persistent=True
        ))
        self.collection = self.client.get_or_create_collection(collection_name)
        self.embedder = SentenceTransformer(EMBEDDING_MODEL)

In [51]:
# Показать статистику
show_collection_stats()

Количество документов: 9
Размерность эмбеддингов: 768
Уникальных пользователей: 1
Средняя длина документа: 40.67


In [52]:
# Поиск документов
search_results = search_documents("Лучший работник", n_results = 10, similarity_threshold = 0.0)
search_results

,Текст,Метаданные,Схожесть,ID
0,Лучший работник месяца - Иван,"{'user': 'Admin', 'source': 'Admin_Notebook', ...",0.620264,doc_Admin_1748051780.354991
1,Зарплаты в нашей компании выше рыночной,"{'user': 'Admin', 'source': 'Admin_Notebook', ...",0.495668,doc_Admin_1748052298.237894
2,Ты модель для помощи связи бизнеса и клиентов,"{'timestamp': '2025-05-24T04:54:05.450723', 's...",0.470214,doc_Admin_1748051645.451723
3,Цель нашей компании - двигать прогресс вперед!,"{'source': 'Admin_Notebook', 'timestamp': '202...",0.441883,doc_Admin_1748051717.199998
4,Нашей компании 10 лет,"{'user': 'Admin', 'timestamp': '2025-05-24T04:...",0.411033,doc_Admin_1748052262.454729
5,В нашей компании работает 35 человек,"{'timestamp': '2025-05-24T04:54:05.450723', 's...",0.348270,doc_Admin_1748051689.32893
6,У нашей компании более 1000 клиентов,"{'source': 'Admin_Notebook', 'timestamp': '202...",0.327573,doc_Admin_1748052053.73207
7,Наша компания называется BitAvantgard,"{'source': 'Admin_Notebook', 'user': 'Admin', ...",0.288570,doc_Admin_1748051852.83045
8,Наша компания специализируется в аналитике дан...,"{'user': 'Admin', 'timestamp': '2025-05-24T04:...",0.223824,doc_Admin_1748052093.313354


In [53]:
full_data = export_to_dataframe()
full_data

,ids,documents,metadatas,embeddings
0,doc_Admin_1748051645.451723,Ты модель для помощи связи бизнеса и клиентов,"{'user': 'Admin', 'source': 'Admin_Notebook', ...","[0.00787985511124134, 0.05739971995353699, -0...."
1,doc_Admin_1748051689.32893,В нашей компании работает 35 человек,"{'source': 'Admin_Notebook', 'timestamp': '202...","[-0.014087633229792118, -0.0010737029369920492..."
2,doc_Admin_1748051717.199998,Цель нашей компании - двигать прогресс вперед!,"{'timestamp': '2025-05-24T04:54:05.450723', 's...","[0.041094522923231125, 0.0788121148943901, -0...."
3,doc_Admin_1748051780.354991,Лучший работник месяца - Иван,"{'source': 'Admin_Notebook', 'timestamp': '202...","[0.029923459514975548, -0.007848449051380157, ..."
4,doc_Admin_1748051852.83045,Наша компания называется BitAvantgard,"{'source': 'Admin_Notebook', 'user': 'Admin', ...","[0.039177510887384415, -0.0028870468959212303,..."
5,doc_Admin_1748052053.73207,У нашей компании более 1000 клиентов,"{'user': 'Admin', 'source': 'Admin_Notebook', ...","[0.020703913643956184, -0.008575145155191422, ..."
6,doc_Admin_1748052093.313354,Наша компания специализируется в аналитике дан...,"{'timestamp': '2025-05-24T04:54:05.450723', 's...","[0.003315544920042157, 0.05738585442304611, -0..."
7,doc_Admin_1748052262.454729,Нашей компании 10 лет,"{'user': 'Admin', 'source': 'Admin_Notebook', ...","[0.03227408975362778, 0.057025160640478134, -0..."
8,doc_Admin_1748052298.237894,Зарплаты в нашей компании выше рыночной,"{'timestamp': '2025-05-24T04:54:05.450723', 'u...","[-0.0335618332028389, 0.041996922343969345, -0..."


In [54]:
#text_to_add = "Зарплаты в нашей компании выше рыночной"
#add_document(text_to_add)

In [55]:
#clean_collection()